In [53]:
import pandas as pd
import sqlalchemy as sqla
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id='3b8a7beb4a434db98620cfae1983cffc',client_secret='a5bc3f14c035496690b540b81c5053f5',))

In [8]:
##Setting Up SQL Engine and Extracting Data
engine = sqla.create_engine('sqlite:///top200data')
pd.read_csv('../data/top200data.csv').to_sql('top200data', engine)

In [9]:
songslist = pd.read_sql('SELECT DISTINCT song, songcode, artist FROM top200data WHERE song & songcode IS NOT NULL ',engine)

In [59]:
def searchfeatures(x,feature):
    '''
    Returns specific feature of a song from Spotify API
    
    Inputs: x=song code, feature: feature of song
    Output: Value of specific feature for the song, nan if no value found
    '''
    try:
        return spotify.audio_features(x)[0][feature]
    except:
        return np.nan
    

In [60]:
#Testing Code on Smaller Dataframe
testdf=pd.DataFrame(columns=['track','dance'])
testdf['track']=songslist['songcode'][0:5]
testdf['dance']=testdf['track'].apply(lambda x: searchfeatures(x,'danceability') )
testdf['energy']=testdf['track'].apply(lambda x: searchfeatures(x,'energy') )
testdf['acousticness']=testdf['track'].apply(lambda x: searchfeatures(x,'acousticness') )

In [36]:
##Filling Features of Each Song to Dataframe
songslist['danceability'] = songslist['songcode'].apply(lambda x: searchfeatures(x,'danceability')) 
songslist['acousticness'] = songslist['songcode'].apply(lambda x: searchfeatures(x,'acousticness')) 
songslist['energy'] = songslist['songcode'].apply(lambda x: searchfeatures(x,'energy')) 

In [4]:
songslist['artistmain'] = songslist['artist'].apply(lambda x: x.split(', ')[0])

In [38]:
songslist.to_csv('../data/songsdata.csv')
